# LCOE Estimator Example

In [ ]:
# @title Clone repository (hidden in Colab)
try:
    import google.colab

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/tannhorn/ele-cost.git
    %cd ele-cost

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tannhorn/ele-cost/blob/main/example.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-black?logo=github)](https://github.com/tannhorn/ele-cost/blob/main/example.ipynb)

This is a simple example walkthrough of how to use the code in this repo to calculate LCOE. Python knowledge is beneficial, of course, but not completely necessary. Most of the cells are self-explanatory even without coding skills and you can just run them and read through. And maybe the rest will spur your desire to learn some Python :-)?

We start by importing the functionality and some other packages we will need.

In [44]:
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp

from lcoe import evaluate as evl

Let's introduce necessary data. They are based on the information presented in [IEA/OECD-NEA report](https://iea.blob.core.windows.net/assets/ae17da3d-e8a5-4163-a3ec-2e6fb0b5677d/Projected-Costs-of-Generating-Electricity-2020.pdf) titled "Projected Costs of Generating Electricity 2020,". They should correspond to a typical 1000 MWe LWR project at pre-2020 prices. 


We assume discount rate of 7% and calculate everything relative to now, taken as year 1.

In [45]:
discount_rate = 0.07
present_year = 1

power = 1000  # MWe
power_kw = power * 1000  # kWe

The project will consist of three phases: construction, operations, and decommissioning. We introduce their durations in years.

In [46]:
construction_duration = 10  # years
operational_lifetime = 60  # years
decommissioning_duration = 10  # years

### Project Costs

The first chunk of costs will be the capital costs. We use the supplied information on overnight construction costs and calculate capital cost per year.

In [47]:
overnight_cost_per_kwe = 8600  # $/kWe, median
overnight_cost = overnight_cost_per_kwe * power_kw

capital_cost_per_year = overnight_cost / construction_duration

Second chunk of costs will be O&M and fuel costs. For that we need to calculate energy production per year.

In [48]:
capacity_factor = 0.85
hours_in_year = 8760
mwh_per_year = hours_in_year * power * capacity_factor

This can now be used to estimae the O&M and fuel costs, those including both front-end and back-end costs.

In [49]:
# o&m
o_and_m_cost_per_kw_year = 80  # $/kwe/year
o_and_m_cost_per_year = o_and_m_cost_per_kw_year * power_kw

# fuel
front_end_fuel_cost_per_mwh = 7  # $/MWh
back_end_fuel_cost_per_mwh = 2.33  # $/MWh
total_fuel_cost_per_mwh = front_end_fuel_cost_per_mwh + back_end_fuel_cost_per_mwh

fuel_cost_per_year = total_fuel_cost_per_mwh * mwh_per_year

Another cost item considered in the IEA report is carbon cost. This is of course not very relevant for a nuclear power plant, but we put the cost item here as well, for completeness.

In [50]:
# carbon
carbon_cost_per_ton = 30  # $/t CO2
carbon_content_per_mwh = 0  # t/MWh
carbon_cost_per_mwh = carbon_cost_per_ton * carbon_content_per_mwh

carbon_cost_per_year = carbon_cost_per_mwh * mwh_per_year

The final cost item we consider is decommissioning, for which an estimate is produced based on the overnight cost and a multiplier factor.

In [51]:
# decommissioning
decommissioning_cost_factor = 0.15
decommissioning_cost = decommissioning_cost_factor * overnight_cost
decommissioning_cost_per_year = decommissioning_cost / decommissioning_duration

### Cash and Revenue Flows

These are all the cost items we need; now we define the timeline of the project, including construction, operation and decommissioning (each of the phases assumed to start immediately after the previous one).

In [52]:
# timeline
construction_start = present_year
construction_end = present_year + construction_duration - 1
operation_start = construction_end + 1
operation_end = operation_start + operational_lifetime - 1
decommissioning_start = operation_end + 1
decommissioning_end = decommissioning_start + decommissioning_duration - 1

The timeline, together with the cost items, will now be used to create cash flows in time. For that, the functionality provided in this repo will be used.

In [53]:
# cash flows
cost_items_rolled = {
    "Capital": (construction_start, construction_end, capital_cost_per_year),
    "O&M": (operation_start, operation_end, o_and_m_cost_per_year),
    "Fuel": (operation_start, operation_end, fuel_cost_per_year),
    "Carbon": (operation_start, operation_end, carbon_cost_per_year),
    "Decommissioning": (
        decommissioning_start,
        decommissioning_end,
        decommissioning_cost_per_year,
    ),
}

# Create the cost_items dictionary from cost_items_rolled
cost_items = evl.create_cost_items(cost_items_rolled)

We will also create a revenue flow; for the purposes of LCOE, that is simply energy generation per year treated as a flow (i.e. it can be discounted).

In [54]:
# Revenue in form of MWh per year
revenue_data = evl.create_cash_flow((operation_start, operation_end, mwh_per_year))

### LCOE

Now, the LCOE can be calculated!

In [55]:
lcoe = evl.calculate_lcoe(cost_items, revenue_data, discount_rate, present_year)
print(f"LCOE with a discount rate of {discount_rate*100:.2f}%: {lcoe:.2f} $/MWh")

LCOE with a discount rate of 7.00%: 133.89 $/MWh


## Effect of Discounting

As you might have noticed, the discount rate only entered the calculation at the very end. Even though the code in `evaluate.py` allows for assigning discount rates to the individual cash flows, it is also possible _not_ to do that and apply a default discount rate at the very end, when net present value is calculated. Therefore, we can for example calculate the LCOE if 0% discounting is assumed (both for costs and for revenue/energy production).

In [56]:
zero_discount_rate = 0
zero_lcoe = evl.calculate_lcoe(
    cost_items, revenue_data, zero_discount_rate, present_year
)
print(
    f"LCOE with a discount rate of {zero_discount_rate*100:.2f}%: {zero_lcoe:.2f} $/MWh"
)

LCOE with a discount rate of 0.00%: 42.21 $/MWh


That is a significant difference! That is due to the fact that the operational life of a nuclear power plant is very long and 7% discounting effectively negates any revenue (and cost) after 20-30 years. This is shown below, where the discount factor is plotted. It is an interactive plot so you can try different discount rates as well. _(don't worry if you don't understand the code, it is basically just plotting)_

In [57]:
# Define the function to calculate the discount factor
def discounting_function(time, discount_rate):
    return 1 / (1 + discount_rate) ** time


# Create time range
time = np.linspace(0, 80, 500)  # Time from 0 to 80 years

# Create the figure
fig = go.Figure()

# Add the initial discounting function
fig.add_trace(
    go.Scatter(
        x=time,
        y=discounting_function(time, discount_rate),
        mode="lines",
        name=f"Discount Rate: {discount_rate*100:.1f}%",
        line=dict(width=2),
    )
)

# Define the slider steps
steps = []
for rate in np.linspace(0.01, 0.1, 10):  # Rates from 1% to 10%
    step = dict(
        method="update",
        label=f"{rate*100:.1f}%",
        args=[
            {"y": [discounting_function(time, rate)]},  # Update y values
            {
                "title": f"Discounting Function Over Time (Discount Rate: {rate*100:.1f}%)"
            },  # Update title
        ],
    )
    steps.append(step)

# Add the slider
fig.update_layout(
    title=f"Discounting Function Over Time (Discount Rate: {discount_rate*100:.1f}%)",
    xaxis_title="Time (Years)",
    yaxis_title="Discount Factor",
    template="plotly_white",
    font=dict(size=14),
    sliders=[
        {
            "active": 6,  # Index of 7% in the slider steps (6th step if range is 1%-10% in 20 increments)
            "currentvalue": {"prefix": "Discount Rate: ", "font": {"size": 14}},
            "steps": steps,
        }
    ],
)

# Show the plot
fig.show()

Seeing the effect of discounting, an interesting question arises. Typically O&M, fuel and decomissioning costs are taken to be negligible for nuclear power plants. Of course, we saw that the LCOE drops rapidly by eliminating the discount rate -- but what does that do to the cost distribution? For that, we use another function from the `evaluate.py`, which uses the cost item dictionary to calculate a dictionary of discounted cash flows -- and this can be done with whichever discount rate we choose!

In [58]:
discounted_costs_default = evl.discount_cash_flows(
    cost_items, discount_rate, present_year
)
discounted_costs_zero = evl.discount_cash_flows(
    cost_items, zero_discount_rate, present_year
)

And this we can now plot side by side!

In [66]:
# Normalize values to 100% for each dictionary
total1 = sum(discounted_costs_default.values())
total2 = sum(discounted_costs_zero.values())

labels = list(discounted_costs_default.keys())  # Extract keys for labels
values1 = [
    v / total1 * 100 for v in discounted_costs_default.values()
]  # Normalize to percentages for dict1
values2 = [
    v / total2 * 100 for v in discounted_costs_zero.values()
]  # Normalize to percentages for dict2

# Create subplots for side-by-side pie charts
subplot_titles = [
    f"Discount rate {discount_rate*100:.0f}%, LCOE {lcoe:.0f} $/MWh",
    f"Discount rate {zero_discount_rate*100:.0f}%, LCOE {zero_lcoe:.0f} $/MWh",
]

fig = sp.make_subplots(
    rows=1,
    cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=subplot_titles,
)

# Add pie chart for dictionary 1
fig.add_trace(
    go.Pie(labels=labels, values=values1, name=subplot_titles[0]), row=1, col=1
)

# Add pie chart for dictionary 2
fig.add_trace(
    go.Pie(labels=labels, values=values2, name=subplot_titles[1]), row=1, col=2
)

# Update layout for better appearance
fig.update_layout(
    title_text="Cost Breakdown for Different Discount Rates",
    title_x=0.5,  # Center the title
    title_y=0.95,
    showlegend=True,
    annotations=[
        dict(
            x=0.20,  # x-position of the title for the first chart
            y=1.05,  # Adjust space above the chart title
            text=subplot_titles[0],
            showarrow=False,
            font=dict(size=14),
        ),
        dict(
            x=0.80,  # x-position of the title for the second chart
            y=1.05,  # Adjust space above the chart title
            text=subplot_titles[1],
            showarrow=False,
            font=dict(size=14),
        ),
    ],
)

# Show the plot
fig.show()